# snnTorch Inference with NIR

## 1. Installation, Imports and Dataset

In [1]:
!pip install snntorch --quiet
!pip install tonic --quiet
!pip install nir --quiet

In [2]:
# imports
import snntorch as snn

import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import tonic

poker_test = tonic.datasets.POKERDVS(save_to='./data', train=False)

In [4]:
import tonic.transforms as transforms
from tonic import DiskCachedDataset

# time_window
frame_transform = tonic.transforms.Compose([tonic.transforms.Denoise(filter_time=10000),
                                            tonic.transforms.ToFrame(
                                            sensor_size=tonic.datasets.POKERDVS.sensor_size,
                                            time_window=1000)
                                            ])

batch_size = 8
cached_testset = DiskCachedDataset(poker_test, transform=frame_transform, cache_path='./cache/pokerdvs/test')
test_loader = DataLoader(cached_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

## 2. Load NIR graph into snnTorch

In [5]:
import nir

In [6]:
nir_model = nir.read("nir_model.nir")

In [7]:
net = snn.import_from_nir(nir_model)

replace rnn subgraph with nirgraph


In [8]:
net.to(device)

GraphExecutor(
  (fc1): Linear(in_features=2450, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=4, bias=True)
  (input): Identity()
  (lif1): Synaptic()
  (lif2): Synaptic()
  (output): Identity()
)

**TODO: add some information about the `nirtorch.GraphExecutor`** 

## 3. Run the model with a single batch of data

The graph executor can run a single forward step. Let's add the recurrence...

In [13]:
def apply(data):
    """
    apply an input data batch to the snntorch model
    """
    state = None
    hid_rec = []
    out = []
    # init/reset membrane voltage for each run
    net.lif1.init_synaptic()
    net.lif2.init_synaptic()
    
    for i, t in enumerate(data):
        z, state = net(t.flatten(1), state)
        print(i, state)
        out.append(z)
        hid_rec.append(state)
    spk_out = torch.stack(out)
    # hid_rec = torch.stack(hid_rec)
    return spk_out, hid_rec

Apply to a batch of data

In [14]:
data, targets = next(iter(test_loader))

spk, hid = apply(data)

# count the number of spikes for each neuron and assess the winner
predictions = spk.sum(axis=0).argmax(axis=-1)
print(f"Predicted classes: {predictions}")
print(f"Actual classes:    {targets}")

TypeError: unsupported operand type(s) for *: 'Tensor' and 'NoneType'

### 4. Measure accuracy for test dataset

In [ ]:
def measure_accuracy(model, dataloader):
  with torch.no_grad():
    # model.eval()  # not needed!
    running_length = 0
    running_accuracy = 0

    for data, targets in iter(dataloader):
      data = data.to(device)
      targets = targets.to(device)

      # forward-pass
      spk_rec, _ = model(data)
      spike_count = spk_rec.sum(0) # batch x num_outputs
      _, max_spike = spike_count.max(1)

      # correct classes for one batch
      num_correct = (max_spike == targets).sum()

      # total accuracy
      running_length += len(targets)
      running_accuracy += num_correct

    accuracy = (running_accuracy / running_length)

    return accuracy.item()

In [ ]:
measure_accuracy(apply, test_loader)